In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="show the raw code.">
''')

In [1]:
import pandas as pd
from ipywidgets import interact, widgets
import boto3
from boto3.s3.transfer import TransferConfig
import os
from urllib.parse import urlparse
from botocore import UNSIGNED
from botocore.config import Config
import example.realTimeNRSNodes as nrs


def content_download(bucket,s3_path,exception):
    
    #Create Session
    session = boto3.Session()
    
    #Initiate S3 Resource anonimous
    s3 = session.resource('s3', config=Config(signature_version=UNSIGNED))

    # Select Your S3 Bucket
    bucket = s3.Bucket(bucket)
    print(s3.Bucket)
    dir = os.getcwd()
    download_path = os.path.join(dir, 'downloads', )
    print(download_path)
    # Iterate All Objects in Your S3 Bucket Over the for Loop
    for s3_object in bucket.objects.filter(Prefix=s3_path):
        path, filename = os.path.split(s3_object.key)
        if exception not in filename:
            #Create sub directories if its not existing
            #os.makedirs(path)
            file_name_with_path=os.path.join(download_path,filename)
            #Download the file in the sub directories or directory if its available. 
            bucket.download_file(s3_object.key, file_name_with_path)
            nrs.load_IMOS_ts_from_local_data(file_name_with_path)
    print('download complete and processing complete')
    

def start_download(object):
    selected_item = dropdown.value
    ind = df.index.where(df['Reference'] == selected_item)
    url = ind.dropna()[0]
    prefix = "prefix="
    path, bucket = url.split(prefix)
    print(path)
    print(bucket)
    str = bucket
    s3_loc = urlparse(str, allow_fragments=False)
    print(s3_loc)
    content_download('imos-data',s3_loc.path.lstrip('/'),'velocity-aggregated-timeseries')
    
source_file = 'sources.csv'
df = pd.read_csv(source_file, delimiter=",",index_col=0)

dropdown = widgets.Dropdown(
    options=df.Reference,
    description='Station:',
    disabled=False,
)

button = widgets.Button(
    description='import data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='import data',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button.on_click(start_download)
display(dropdown)
display(button)

ModuleNotFoundError: No module named 'boto3'